# 1D Spectral analysis

In this tutorial, we present the basics steps for a 1D spectral analysis. The main aim is to perform a spectral analysis of a point-like source. A spectral analysis consist in stacking all events in the region of interest. Doing this we forget about the spatial information of the source. This is why the 1D analysis is best suited for point-like sources. 

To estimate the background, we will use off regions taken on the data using reflected regions. 

## 1. Crab Nebula

### 1.1 Full containment IRFs: H.E.S.S. data

We are going to analyse the data from the High Energy Stereoscopic System (H.E.S.S.) towards the Crab nebula.  

Let's start with some basic imports:

In [ ]:
from gammapy.data import DataStore
from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from gammapy.makers import SpectrumDatasetMaker, ReflectedRegionsBackgroundMaker, SafeMaskMaker, WobbleRegionsFinder
from gammapy.datasets import SpectrumDataset, Datasets
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel, ExpCutoffPowerLawSpectralModel, LogParabolaSpectralModel
from gammapy.modeling.selection import select_nested_models
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator
from gammapy.visualization import plot_spectrum_datasets_off_regions

from astropy.coordinates import SkyCoord
import astropy.units as u
from regions import CircleSkyRegion, PointSkyRegion

import matplotlib.pyplot as plt
import numpy as np

Create a DataStore instance using the following path to H.E.S.S. data: `$GAMMAPY_DATA/hess-dl3-dr1`.

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")

Get the Crab coordinates using astropy `SkyCoord`.

In [ ]:
target_position = SkyCoord.from_name("Crab")

To select the relevant runs, we can use the `obs_table` associated to the `DataStore`.

In [ ]:
obs_table = data_store.obs_table

Using `select_sky_circle` method of `obs_table`, select run that are at most at 2.2 deg from the Crab Nebula.

In [ ]:
obs_table = obs_table.select_sky_circle(target_position, 2.2*u.deg)

We can then get the observations using the `get_observations` method of `DataStore`.

In [ ]:
observations = data_store.get_observations(obs_table["OBS_ID"])

In [ ]:
print(observations)

Define a energy axis using `MapAxis.from_energy_bounds` between 0.1 and 40 TeV. 

Then define a relevant true energy axis. Remember that the true energy axis should have more bins that the reco energy axis and over a wider range. 

In [ ]:
energy_axis = MapAxis.from_energy_bounds(0.1, 40, 10, per_decade=True, unit="TeV")

energy_axis_true = MapAxis.from_energy_bounds(
    0.01, 100, 20, unit="TeV", name="energy_true", per_decade=True
)

Define a region of interest:

In [ ]:
on_region = CircleSkyRegion(target_position, 0.1*u.deg)

For the crab, we have to create a exclusion mask. 

In [ ]:
exclusion_region = CircleSkyRegion(
    center=SkyCoord(183.604, -8.708, unit="deg", frame="galactic"),
    radius=0.5 * u.deg,
)

Plot the exclusion mask:

In [ ]:
skydir = target_position.galactic
geom = WcsGeom.create(
    npix=(150, 150), binsz=0.05, skydir=skydir, proj="TAN", frame="icrs"
)

exclusion_mask = ~geom.region_mask([exclusion_region])
ax = exclusion_mask.plot()
ax.scatter(target_position.ra, target_position.dec, transform=ax.get_transform("icrs"), label="Crab Nebula")
plt.show()

Create a geometry using `RegionGeom`.

In [ ]:
geom = RegionGeom.create(region=on_region, axes=[energy_axis])

Create an empty `SpectrumDataset`.

In [ ]:
dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

Create the Maker instance that we are going to need for the data reduction.

`SpectrumDatasetMaker` will "fill" the dataset with reduces IRFs and counts.

`ReflectedRegionsBackgroundMaker` will get the off counts from reflected regions.

`SafeMaskMaker` will define the safe mask for each dataset.

In [ ]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "exposure", "edisp"]
)
bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)
safe_mask_maker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

Run the data reduction loop. Order matters (dataset_maker, bkg, safe_mask)

In [ ]:
datasets = Datasets()

for obs, obs_id in zip(observations, observations.ids):
    dataset = dataset_maker.run(dataset_empty.copy(name=obs_id), obs)
    dataset_on_off = bkg_maker.run(dataset, obs)
    dataset_on_off = safe_mask_maker.run(dataset_on_off, obs)
    datasets.append(dataset_on_off)

print(datasets)

Parallel version of the reduction loop

```python
makers = [dataset_maker, bkg_maker, safe_mask_maker]  # the order matters
datasets_maker = DatasetsMaker(makers, stack_datasets=False, n_jobs=6)
datasets = datasets_maker.run(dataset_empty, observations)
```

Info table:

In [ ]:
info_table = datasets.info_table(cumulative=True)

In [ ]:
info_table

Plot the reflected regions:

In [ ]:
plt.figure()
ax = exclusion_mask.plot()
on_region.to_pixel(ax.wcs).plot(ax=ax, edgecolor="k")
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets)
plt.show()


Stack the dataset using `stack_reduce`.

In [ ]:
stacked = datasets.stack_reduce(name="stacked")

Create a power-law using the `PowerLawSpectralModel`.

In [ ]:
spectral_model = PowerLawSpectralModel(
    index=2, 
    amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), 
    reference=1*u.TeV
)

Create a `SkyModel`.

In [ ]:
sky_model = SkyModel(spectral_model=spectral_model, name="Crab")

Fit the model to the stacked dataset.

In [ ]:
stacked.models = sky_model

In [ ]:
fit = Fit()
result = fit.run(datasets=stacked)

In [ ]:
print(result)

In [ ]:
stacked.models

Print the residuals:

In [ ]:
stacked.plot_residuals_spectral(method="diff/sqrt(model)")

Extract flux points:

In [ ]:
fp = FluxPointsEstimator(
    energy_edges=energy_axis.edges, 
    source="Crab", 
    selection_optional="all"
).run(datasets=stacked)

Plot the model and flux points.

In [ ]:
sed_type = "e2dnde"
energy_bounds = [0.4, 40]*u.TeV

stacked.models[0].spectral_model.plot(energy_bounds=energy_bounds, sed_type=sed_type, label="Best fit")
stacked.models[0].spectral_model.plot_error(energy_bounds=energy_bounds, sed_type=sed_type)
fp.plot(sed_type=sed_type, label="Flux Points")

#### Model significance

Exercise: Is a power-law the best model to describe the data? 

To answer this question, we propose to use a likelihood ratio test (see Jonathan lecture) test both a `LogParabolaSpectralModel` and an `ExpCutoffPowerLawSpectralModel`.

Are those two models nested models to a power-law respectively?

Can we apply Wilks' theorem? 

Knowing that the "total stat" of the fit result is $-2\ln(\mathcal{L})$, where $\mathcal{L}$ is the likelihood, compute the significance of the exponentially cutoff power-law and the log-parabola against the power-law. 

Can you do the same likelihood ratio test to distinguish the exponentially cutoff power-law and the log-parabola?

In [ ]:
spectral_model_exp = ExpCutoffPowerLawSpectralModel(
    index=2, 
    amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), 
    reference=1*u.TeV,
    lambda_ = (1/10)/u.TeV,
)

In [ ]:
sky_model_exp = SkyModel(spectral_model=spectral_model_exp, name="Crab exp")

In [ ]:
stacked.models = sky_model_exp

In [ ]:
fit = Fit()
result_exp = fit.run(datasets=stacked)

In [ ]:
print(result_exp)

In [ ]:
stacked.models

In [ ]:
stacked.plot_residuals_spectral(method="diff/sqrt(model)")

In [ ]:
h0 = result.optimize_result.total_stat
h1 = result_exp.optimize_result.total_stat
likelihood_ratio_ts = -(h1 - h0)
likelihood_ratio_ts

In [ ]:
np.sqrt(likelihood_ratio_ts)

## Alternative approach for evaluate nested models using Gammapy API
Alternatively, Gammapy also allows evaluating two nested models using [`gammapy.modeling.select_nested_models`](https://docs.gammapy.org/2.0/api/gammapy.modeling.select_nested_models.html)

If there is no cutoff (null hypothesis) then the spectral cutoff (`lambda`) is zero. So that is the parameter we input in this function along with the null value against we are hypothesis testing.

**Note:** This is useful for testing the significance of adding model components, such as a spectral cutoff or a source detection. It assumes the two models are nested — the null hypothesis is a special case of the alternative with fixed parameters.

In [ ]:
result_nested_models = select_nested_models(
    stacked,
    parameters=[sky_model_exp.spectral_model.lambda_],
    null_values=[0],
)
result_nested_models["ts"]

## Flux points estimation

In [ ]:
fp_exp = FluxPointsEstimator(
    energy_edges=energy_axis.edges, 
    source="Crab exp", 
    selection_optional="all"
).run(datasets=stacked)

In [ ]:
sed_type = "e2dnde"
energy_bounds = [0.4, 40]*u.TeV

stacked.models[0].spectral_model.plot(energy_bounds=energy_bounds, sed_type=sed_type, label="Best fit")
stacked.models[0].spectral_model.plot_error(energy_bounds=energy_bounds, sed_type=sed_type)
fp.plot(sed_type=sed_type, label="Flux Points")

plt.xlim(0.4, 40)

In [ ]:
spectral_model_log = LogParabolaSpectralModel(
    alpha=2, 
    amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), 
    reference=1*u.TeV,
    beta=1,
)

In [ ]:
sky_model_log = SkyModel(spectral_model=spectral_model_log, name="Crab log")

In [ ]:
stacked.models = sky_model_log

In [ ]:
fit = Fit()
result_log = fit.run(datasets=stacked)

In [ ]:
print(result_exp)

In [ ]:
stacked.models

In [ ]:
stacked.plot_residuals_spectral(method="diff/sqrt(model)")

In [ ]:
h0 = result.optimize_result.total_stat
h1 = result_log.optimize_result.total_stat
likelihood_ratio_ts = -(h1 - h0)

In [ ]:
np.sqrt(likelihood_ratio_ts)

In [ ]:
fp_log = FluxPointsEstimator(
     energy_edges=energy_axis.edges, 
    source="Crab log", 
    selection_optional="all"
).run(datasets=stacked)

In [ ]:
sed_type = "e2dnde"
energy_bounds = [0.4, 40]*u.TeV

stacked.models[0].spectral_model.plot(energy_bounds=energy_bounds, sed_type=sed_type, label="Best fit")
stacked.models[0].spectral_model.plot_error(energy_bounds=energy_bounds, sed_type=sed_type)
fp.plot(sed_type=sed_type, label="Flux Points")

plt.xlim(0.4, 40)

### 1.2 Point-like IRFs

#### Redo the Crab nebulae analysis using "Point-like" observations from MAGIC

For that you will have to get the data from `$GAMMAPY_DATA/magic/rad_max/data/`. 

To get the observations, `required_irf="point-like"` has to be passed to `DataStore.get_observations`.

Also note that MAGIC energy threshold is lower that H.E.S.S.. You can start your dataset reco energy axis at 50 GeV. Of course, you will have to adapt you true energy in consequences.

Finally, using "point-like" IRFs, it is better to use Wobble regions for the off counts. You can define region_finder of `ReflectedRegionsBackgroundMaker` to `ReflectedRegionsBackgroundMaker`. 

In [ ]:
data_store_magic = DataStore.from_dir("$GAMMAPY_DATA/magic/rad_max/data/")

In [ ]:
observations = data_store_magic.get_observations(required_irf="point-like")

In [ ]:
on_region = PointSkyRegion(target_position)

In [ ]:
energy_axis = MapAxis.from_energy_bounds(
    40, 2e4, nbin=5, per_decade=True, unit="GeV", name="energy"
)
energy_axis_true = MapAxis.from_energy_bounds(
    10, 1e5, nbin=10, per_decade=True, unit="GeV", name="energy_true"
)

In [ ]:
geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

dataset_maker = SpectrumDatasetMaker(
    selection=["counts", "exposure", "edisp"]
)

region_finder = WobbleRegionsFinder(n_off_regions=3)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

safe_mask_maker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

In [ ]:
datasets = Datasets()

for obs, obs_id in zip(observations, observations.ids):
    dataset = dataset_maker.run(dataset_empty.copy(name=obs_id), obs)
    dataset_on_off = bkg_maker.run(dataset, obs)
    dataset_on_off = safe_mask_maker.run(dataset_on_off, obs)
    datasets.append(dataset_on_off)

print(datasets)

In [ ]:
stacked = datasets.stack_reduce(name="stacked")

In [ ]:
plt.figure()
ax = exclusion_mask.plot()
on_region.to_pixel(ax.wcs).plot(ax=ax)
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets)
plt.show()

In [ ]:
spectral_model_exp = ExpCutoffPowerLawSpectralModel(
    index=2, 
    amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), 
    reference=1*u.TeV,
    lambda_ = (1/10)/u.TeV,
)

In [ ]:
sky_model_exp = SkyModel(spectral_model=spectral_model_exp, name="Crab exp")

In [ ]:
stacked.models = sky_model_exp

In [ ]:
for d in datasets:
    d.mask_fit = dataset.counts.geom.energy_mask(20*u.GeV, 20*u.TeV)

In [ ]:
fit = Fit()
result_exp = fit.run(datasets=stacked)

In [ ]:
print(result_exp)

In [ ]:
print(stacked.models)

In [ ]:
stacked.plot_residuals_spectral(method="diff/sqrt(model)")

In [ ]:
fp_exp = FluxPointsEstimator(
     energy_edges=energy_axis.edges, 
    source="Crab exp", 
    selection_optional="all"
).run(datasets=stacked)

In [ ]:
sed_type = "e2dnde"
energy_bounds = [0.08, 20]*u.TeV

stacked.models[0].spectral_model.plot(energy_bounds=energy_bounds, sed_type=sed_type, label="Best fit")
stacked.models[0].spectral_model.plot_error(energy_bounds=energy_bounds, sed_type=sed_type)
fp_exp.plot(sed_type=sed_type, label="Flux Points")

plt.xlim(0.08, 20)
plt.ylim(2e-12, 2e-10)

## Open problem: RX J1713.7-3946 1D

This morning we did the analysis of `RX J1713.7-3946` using the 3D analysis. Do the same analysis with the using a spectral analysis.

Questions to ask yourself: 
 - What is the size of the region of interest for this source ?
 - Do you need an exclusion mask ?
 - Are your off regions correctly measured ?
 - Conclude !

In [ ]:
target_position = SkyCoord.from_name("RX J1713.7-3946")

In [ ]:
obs_table = data_store.obs_table

In [ ]:
obs_table = obs_table.select_sky_circle(target_position, 2.2*u.deg)

In [ ]:
observations = data_store.get_observations(obs_table["OBS_ID"])

In [ ]:
print(observations)

In [ ]:
energy_axis = MapAxis.from_energy_bounds(0.3, 10.0, 15, unit="TeV")

energy_axis_true = MapAxis.from_energy_bounds(
    0.1, 20, 20, unit="TeV", name="energy_true"
)

In [ ]:
on_region = CircleSkyRegion(target_position, 0.5*u.deg)

In [ ]:
geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "exposure", "edisp"]
)
bkg_maker = ReflectedRegionsBackgroundMaker()
safe_mask_maker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

In [ ]:
datasets = Datasets()

for obs, obs_id in zip(observations, observations.ids):
    dataset = dataset_maker.run(dataset_empty.copy(name=obs_id), obs)
    dataset_on_off = bkg_maker.run(dataset, obs)
    dataset_on_off = safe_mask_maker.run(dataset_on_off, obs)
    datasets.append(dataset_on_off)

print(datasets)

In [ ]:
info_table = datasets.info_table(cumulative=True)

In [ ]:
info_table

In [ ]:
stacked = datasets.stack_reduce(name="stacked")

In [ ]:
spectral_model = PowerLawSpectralModel(
    index=2, 
    amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), 
    reference=1*u.TeV
)

In [ ]:
sky_model = SkyModel(spectral_model=spectral_model, name="RX")

In [ ]:
stacked.models = [sky_model]

In [ ]:
fit = Fit()
result = fit.run(datasets=stacked)

In [ ]:
print(result)

In [ ]:
print(stacked)

In [ ]:
stacked.models[0].spectral_model.plot_error(energy_bounds=[0.3, 10]*u.TeV, sed_type="e2dnde")

In [ ]:
spectral_model_3d = PowerLawSpectralModel(
    index=1.913,
    amplitude="2.30e-11 TeV-1 cm-2 s-1",
    reference="1 TeV",
)
spectral_model_3d.index.error = 0.04
spectral_model_3d.amplitude.error = 9.0e-13

In [ ]:
energy_bounds=[0.3, 10]*u.TeV
sed_type="e2dnde"

stacked.models[0].spectral_model.plot(energy_bounds=energy_bounds, sed_type=sed_type, label="1D")
stacked.models[0].spectral_model.plot_error(energy_bounds=energy_bounds, sed_type=sed_type)

spectral_model_3d.plot(energy_bounds=energy_bounds, sed_type=sed_type, label="3D")
spectral_model_3d.plot_error(energy_bounds=energy_bounds, sed_type=sed_type)

plt.legend()